In [1]:
from mlforecast.lgb_cv import LightGBMCV
from utilsforecast.data import generate_series

# Generate synthetic time series data
df = generate_series(
    n_series=5,
    freq='D',
    min_length=700,
    max_length=700,
    equal_ends=True
)

cv = LightGBMCV(
    freq='D',
    # target_transforms=[Differences([24])],
    lags=range(1, 2)
)

cv_results = cv.fit(
    df,
    h=48,
    n_windows=4,
    eval_every=10,
    early_stopping_evals=2,
    early_stopping_pct=0.01,
    compute_cv_preds=True
)

11
None
     unique_id         ds         y      lag1
1            0 2000-01-02  1.357595  0.274407
2            0 2000-01-03  2.301382  1.357595
3            0 2000-01-04  3.272442  2.301382
4            0 2000-01-05  4.211827  3.272442
5            0 2000-01-06  5.322947  4.211827
...        ...        ...       ...       ...
3303         4 2001-05-18  6.461616  5.250618
3304         4 2001-05-19  0.309320  6.461616
3305         4 2001-05-20  1.104413  0.309320
3306         4 2001-05-21  2.341215  1.104413
3307         4 2001-05-22  3.384181  2.341215

[2535 rows x 4 columns]
22
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 2535, number of used features: 1
     unique_id         ds         y      lag1
1            0 2000-01-02  1.357595  0.274407
2            0 2000-01-03  2.30138

In [1]:
from mlforecast.lgb_cv import LightGBMCV
from utilsforecast.data import generate_series
import numpy as np

# Generate synthetic time series data
df = generate_series(
    n_series=5,
    freq='D',
    min_length=700,
    max_length=700,
    equal_ends=True
)

# Generate weights: for instance, give a few series higher weights
num_samples = df.shape[0]
np.random.seed(42)  # For reproducibility
weights = np.random.pareto(a=2.0, size=num_samples)  # Generate a heavy-tailed distribution
weights = weights / np.sum(weights) * num_samples  # Normalize weights

# Add the weights to the DataFrame
df['weightsxx'] = weights

# Initialize LightGBMCV
cv = LightGBMCV(
    freq='D',
    # target_transforms=[Differences([24])],
    lags=range(1, 2)
)

# Fit the model with the weights
cv_results = cv.fit(
    df=df,
    h=48,
    n_windows=4,
    eval_every=10,
    early_stopping_evals=2,
    early_stopping_pct=0.01,
    compute_cv_preds=True,
    weight_col='weightsxx',  # Specify the weight column
    static_features=[]
)

# Display the results
print(cv_results)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000053 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 2535, number of used features: 2
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000019 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 2775, number of used features: 2
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000213 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 3015, number of used features: 2
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of test

[LightGBM] [Fatal] The number of features in data (1) is not the same as it was in training data (2).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.


LightGBMError: The number of features in data (1) is not the same as it was in training data (2).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [4]:
import numpy as np
from mlforecast.utils import generate_daily_series
from mlforecast import MLForecast
import lightgbm as lgb

# Generate data with a weight column
series = generate_daily_series(2)
series['weight'] = np.random.default_rng(seed=0).random(series.shape[0])

# Create MLForecast object
fcst = MLForecast(
    models={'lgbm': lgb.LGBMRegressor(verbosity=-1)},
    freq='D',
    lags=[1],
    date_features=['dayofweek'],
)

# Fit with weight_col
fcst.fit(series, weight_col='weight').predict(1)

# Cross validation with weight_col
fcst.cross_validation(series, n_windows=2, h=1, weight_col='weight')

,unique_id,ds,cutoff,y,lgbm
0,id_0,2000-08-08,2000-08-07,3.436325,3.242790
1,id_1,2000-04-05,2000-04-04,2.430276,2.075247
2,id_0,2000-08-09,2000-08-08,4.136771,4.239010
3,id_1,2000-04-06,2000-04-05,3.363522,3.436962
